# Ranking using regression

Implementation of pairwise ranking through regression, using xgboost.

****

**Plan:**
- Generate data
    - Features: like previously (random normal)
    - Labels: uniform at random from (20, 40, 80, 100, 120)
        - **This takes a long time: would need to replicate training data 50 (?) times on average**
- Setup training data
    - Xi, Xj, Xi - Xj, yi - yj
    - For all possible pairs of feaure rows
    - **(?) Repeat training data |yi-yj| times (?)**
        - Do this for classification...
    - Arrange labels in m x m matrix (?)
        - For later step
        - **No -- only applicable for making predictions**
- Learn xgboost regressor on training data
- Output predictions from xgboost regressor
    - Store predictions in m x m matrix
    - Apply sigmoid (logit) function to scores in predictions matrix
    - Sum across rows (?) in predictions matrix to get final score for each row
    - Use each final score to rank
    
****

**Questions:**
- Repeat training data |yi-yj| times for regression, or just for classification?
- Sampled labels uniformly at random from (1, 2, 3, 4, 5) rather than (20, 40, 80, 100, 120) since the later would require many more duplications. Is this fine?
- Used m = 1000 (1000 training examples). Is this fine?
- I did not train/test split the data. Is this fine?

In [1]:
import numpy as np
from xgboost import XGBRegressor
from scipy.special import expit  # Logistic function
from rank_metrics import ndcg_at_k

In [2]:
# For reproducible results
np.random.seed(1)

# Dimensions of generated data
m = 1000    # Training examples
n = 5       # Features

In [3]:
# Generate features randomly (gaussian)
X = np.random.normal(loc=0, scale=1, size=(m, n))

# Generate labels from uniform distribution with given values
# y_values = [20, 40, 80, 100, 120]
y_values = [1, 2, 3, 4, 5]
y = np.random.choice(a=y_values, size=m, replace=True)

In [4]:
# Will repeat each row (pairwise feature comparison) |yi - yj| times
# First construct NaN array, then fill in (potentially faster)
# When constructing NaN array, fill in with 'worst-case' (every row repeated max times)
max_diff = max(y_values) - min(y_values)
train = np.full(shape=(m * m * max_diff, 3 * n + 3), fill_value=np.nan)
idx = 0

# First feature
for i in range(m):
    
    # Second feature
    for j in range(m):
        
        y_diff = y[i] - y[j]

        # Repeat each row |yi - yj| times, need to include at least once
        for k in range(int(abs(y_diff)) + 1):
            
            # Store first index, second index, first feature, second feature, 
            # feature difference, and label difference
            row_new = np.hstack(([i, j], X[i, :], X[j, :], X[i, :] - X[j, :], y_diff))
            train[idx] = row_new
            idx += 1
    
    # See progress
    if i % (m / 10) == 0: print(i)

0
100
200
300
400
500
600
700
800
900


In [5]:
# Make sure shapes match up
idx, train[~np.isnan(train[:, 0])].shape

(2647956, (2647956, 18))

In [6]:
# Only store part of array filled in thru iteration
train = train[~np.isnan(train[:, 0])]

# Store new features and labels to actually train model
X_new = train[:, 2:-1]
y_new = train[:, -1]

In [7]:
# Default parameters except max_depth (default 3), n_estimators (default 100), 
# objective (default: 'reg:linear', deprecated)
xgbr = XGBRegressor(max_depth=6, 
                    learning_rate=0.1,
                    n_estimators=10,
                    objective='reg:squarederror')
xgbr.fit(X_new, y_new)

/Users/Ashtekar15/anaconda3/lib/python3.6/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [8]:
# To make predictions, get unique training features along with indices i, j 
# as first two columns
X_idx = train[:, :-1]
X_idx = np.unique(X_idx, axis=0)

# Make predictions on data not including indices
y_pred = xgbr.predict(X_idx[:, 2:])

In [9]:
# Create pairwise ranking matrix to fill in later
mat = np.full(shape=(m, m), fill_value=np.nan)

# Fill in ranking matrix
for i in range(m * m):
    mat[int(X_idx[i, 0]), int(X_idx[i, 1])] = y_pred[i]

# Apply logistic function 
mat = expit(mat)

In [10]:
# Values along diagonal should be about 0.5 (error due to model)
diag = np.diagonal(mat)
diag[:100]

array([0.53483967, 0.51457834, 0.53391171, 0.55346845, 0.55940523,
       0.5401626 , 0.55142085, 0.55136268, 0.55910923, 0.55502063,
       0.54611872, 0.52765327, 0.5401626 , 0.57655319, 0.55091207,
       0.54124595, 0.53649706, 0.56040828, 0.53483967, 0.53483967,
       0.53698847, 0.54217587, 0.55346845, 0.55091207, 0.58553529,
       0.53483967, 0.55091207, 0.55820419, 0.55091207, 0.55215229,
       0.42844657, 0.55142085, 0.5401626 , 0.45144239, 0.46161585,
       0.53698847, 0.53973858, 0.52145534, 0.59924975, 0.56477312,
       0.48707164, 0.49701848, 0.55142085, 0.55012457, 0.51691972,
       0.54889208, 0.54054971, 0.56505871, 0.51428816, 0.54054971,
       0.50486452, 0.53698847, 0.53483967, 0.52556295, 0.51625583,
       0.58151676, 0.55820419, 0.54217587, 0.54748589, 0.5401626 ,
       0.54889208, 0.54791149, 0.57886133, 0.54611872, 0.5401626 ,
       0.53741646, 0.56304411, 0.48789411, 0.51461653, 0.53698847,
       0.54791149, 0.55142085, 0.49913488, 0.55147246, 0.53741

In [11]:
# Stats for entire matrix (to compare with diagonal)
np.mean(mat), np.std(mat), np.min(mat), np.max(mat)

(0.5421178425025636,
 0.08675644746204147,
 0.13441176174041777,
 0.894677791903511)

In [12]:
# Stats for diagonal of matrix, should be more closely grouped around 0.5 vs entire matrix
np.mean(diag), np.std(diag), np.min(diag), np.max(diag)

(0.5422277349882161,
 0.025801931383629677,
 0.4139760064306653,
 0.6607326172740777)

In [13]:
# Sum across rows to get 'power' of each individual training example
# (total rank score relative to the rest of the training examples)
scores = np.sum(mat, axis=0)
ranking = np.argsort(scores)
ranking[:10]

array([853, 389,  40, 526, 879, 609, 136,  50, 587, 925])

In [14]:
# Stats for scores
np.mean(scores), np.std(scores), np.min(scores), np.max(scores)

(542.1178425025632, 63.36784558275654, 224.6707733421218, 819.3795738262625)

In [15]:
# Confirm max and min ranking is consistent with above
scores[ranking[0]], scores[ranking[-1]]

(224.6707733421218, 819.3795738262625)

In [16]:
# See if ranking results make sense w.r.t. original labels
y[ranking]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 4, 4,
       5, 5, 2, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 5, 4, 5, 4, 4, 4, 3, 4,
       4, 4, 5, 4, 4, 4, 5, 3, 2, 5, 4, 5, 2, 5, 5, 5, 4, 5, 5, 4, 3, 4,
       5, 2, 4, 5, 3, 5, 3, 5, 5, 5, 2, 5, 5, 5, 4, 5, 5, 5, 3, 5, 2, 4,
       5, 5, 4, 4, 1, 4, 5, 5, 4, 3, 5, 5, 3, 4, 5, 5, 4, 5, 3, 5, 3, 2,
       5, 3, 5, 5, 4, 1, 4, 5, 4, 3, 5, 4, 3, 1, 3, 5, 4, 3, 3, 5, 5, 5,
       5, 5, 5, 4, 5, 5, 3, 4, 2, 5, 5, 4, 4, 5, 5, 5, 5, 5, 3, 4, 4, 3,
       3, 5, 4, 3, 4, 4, 4, 5, 3, 4, 5, 5, 5, 2, 1, 3, 4, 2, 3, 5, 5, 4,
       4, 3, 3, 4, 3, 5, 5, 5, 4, 4, 4, 4, 5, 3, 4, 5, 2, 5, 2, 2, 2, 5,
       5, 1, 1, 4, 2, 3, 3, 5, 2, 5, 4, 4, 4, 4, 1, 4, 4, 5, 4, 5, 2, 5,
       1, 4, 2, 3, 3, 5, 1, 4, 2, 4, 5, 4, 3, 5, 4, 4, 4, 5, 5, 5, 2, 4,
       5, 2, 5, 3, 2, 4, 4, 4, 3, 4, 5, 3, 5, 3, 2, 3, 4, 4, 5, 5, 5, 5,
       4, 4, 1, 5, 4, 5, 3, 4, 2, 3, 3, 2, 3, 4, 3, 5, 5, 4, 4, 5, 4, 5,
       5, 4, 5, 4, 2, 5, 3, 2, 5, 3, 5, 4, 3, 5, 4,

In [17]:
# NDCG metric, 0 is worst and 1 is best (?)
r = y[ranking]
ndcg_at_k(r=r, k=m)  # 'method' parameter in function - has to do with weighting (?)

0.9739155100075407